<a href="https://colab.research.google.com/github/emilia-uncc/IntroML/blob/main/assignment5/Assignment5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.optim as optim


#mounting google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
file_path = '/content/drive/My Drive/DataSets/Housing.csv'
housing = pd.DataFrame(pd.read_csv(file_path))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [50]:
housing.isnull().sum()*100/housing.shape[0]

price               0.0
area                0.0
bedrooms            0.0
bathrooms           0.0
stories             0.0
mainroad            0.0
guestroom           0.0
basement            0.0
hotwaterheating     0.0
airconditioning     0.0
parking             0.0
prefarea            0.0
furnishingstatus    0.0
dtype: float64

In [51]:
varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0})

# Applying the function to the housing list
housing[varlist] = housing[varlist].apply(binary_map)
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished


In [52]:
## Generate a list of integer positions for the columns you want to select
#area, bedrooms, bathrooms, stories, parking
#selected_columns_indices = [i for i in range(2,32)]  # Selects the first 30 columns
selected_columns_indices = [1, 2, 3, 4, 10]
print(selected_columns_indices)
X = housing.iloc[:, selected_columns_indices].values
#M = 1, E = 0
Y = housing.iloc[:, 0].values
#print(X)
#print(Y)

[1, 2, 3, 4, 10]


In [53]:
np.random.seed(0)
#df_train = training set, #df_test = testset
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

In [54]:
#scale dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
print(X_train)

[[-0.71677205 -1.29437561 -0.57330726 -0.93314164 -0.81914879]
 [-0.53893631 -1.29437561 -0.57330726 -0.93314164 -0.81914879]
 [-0.98820554 -1.29437561 -0.57330726 -0.93314164 -0.81914879]
 ...
 [ 0.28004406  0.05251643  1.4755613   2.50502532  0.32555914]
 [-0.72613182  0.05251643 -0.57330726 -0.93314164  0.32555914]
 [ 0.67783452  0.05251643 -0.57330726 -0.93314164 -0.81914879]]


In [55]:
#------------------functions general-----------------------------------------



# Define a custom loss function (mean squared error)
def loss_fn(y_pred, target):
    return torch.mean((y_pred - target) ** 2)

#------------------functions linear model-------------------------------------
# Define a custom multi-dimensional linear model as a tensor
def multi_linear_model(input_size, output_size):
    weights = torch.rand(output_size, input_size, dtype=torch.float32)   # Initialize weights with small random values
    weights.requires_grad = True  # Enable gradient tracking
    return weights
    #return torch.zeros((output_size, input_size), dtype=torch.float32, requires_grad=True)

#SGD and Adam
def training_loop_linear(n_epochs, optimizer, params, t_u, t_c):
  for epoch in range(1, n_epochs +1):
    t_p = model(t_u, *params) #calc output
    loss = loss_fn(t_p, t_c) #calc loss

    optimizer.zero_grad() # zero the gradients
    loss.backward() #calc gradients of the params
    optimizer.step() #calc the gradients

    if epoch % 500 == 0:
      print(f"Epoch: {epoch}, Loss: {float(loss)}")
      #print('Epoch %d, Loss %f' % (epoch, float(loss)))

  return params

# Define your training loop
def training_loop(n_epochs, optimizer, weights, inputs, targets, test_set, test_target):
    losses = []
    val_losses = []

    for epoch in range(1, n_epochs + 1):

        predictions = torch.matmul(inputs, weights.t())
        loss = loss_fn(predictions, targets)
        pred_test = torch.matmul(test_set, weights.t())
        loss_test = loss_fn(pred_test, test_target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 500 == 0:
            print(weights)
            print(f"Epoch: {epoch}, Loss: {loss.item()}, val_loss: {loss_test.item()}")
            losses.append(loss.item())
            val_losses.append(loss_test.item())

    return weights, losses, val_losses


In [56]:
#arrays into tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

Y_train_tensor = torch.tensor(Y_train, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test, dtype=torch.float32)

# Example usage with multi-dimensional input data:
input_size = 5  # Change this to the number of input features
output_size = 1  # Output size is 1 for regression

weights = multi_linear_model(input_size, output_size)
#optimizer = optim.SGD([weights], lr=0.0001)
n_epochs = 5000
optimizer = optim.Adam([weights], lr = 0.0000001)

weights, losses, val_losses = training_loop(n_epochs, optimizer, weights, X_train_tensor, Y_train_tensor, X_test_tensor, Y_test_tensor)

test_predictions = torch.matmul(X_test_tensor, weights.t())
#print(test_predictions)
print(weights)
print(losses)
print(val_losses)
#630995459954

tensor([[0.3083, 0.0873, 0.9778, 0.6497, 0.9004]], requires_grad=True)
Epoch: 500, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3083, 0.0873, 0.9778, 0.6497, 0.9003]], requires_grad=True)
Epoch: 1000, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3083, 0.0872, 0.9777, 0.6496, 0.9003]], requires_grad=True)
Epoch: 1500, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3082, 0.0872, 0.9776, 0.6495, 0.9002]], requires_grad=True)
Epoch: 2000, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3082, 0.0871, 0.9776, 0.6495, 0.9001]], requires_grad=True)
Epoch: 2500, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3081, 0.0871, 0.9775, 0.6494, 0.9001]], requires_grad=True)
Epoch: 3000, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3081, 0.0870, 0.9775, 0.6494, 0.9000]], requires_grad=True)
Epoch: 3500, Loss: 26469979914240.0, val_loss: 25189242896384.0
tensor([[0.3080, 0.0870, 0.9774, 0.6493, 0.9000]], requi